# Import



In [4]:
### Import des bibliothèques principales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from yellowbrick.classifier import ClassificationReport

### Configuration des options pour les graphiques
%matplotlib inline
sns.set(style="whitegrid")


## Merged data

In [26]:

### Charger les datasets
# dataframe depuis kaggle
heart_data_1 = pd.read_csv('../heart_disease_uci.csv')
# dataframe depuis source
heart_data_2 = pd.read_csv('../HeartDiseaseUCI.csv')

### Afficher un aperçu des données
display(heart_data_1.head())
display(heart_data_2.head())

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,1,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,2,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,3,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,4,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,5,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


In [6]:
print(f"heart data 1 shape{heart_data_1.shape}")
print(f"heart data 2 shape{heart_data_2.shape}")

heart data 1 shape(920, 16)
heart data 2 shape(303, 15)


In [ ]:


# Step 1: Rename specific columns in heart_data_1 to match heart_data_2
# Renaming 'id' to 'Unnamed: 0' and 'thalch' to 'thalach'
heart_data_1 = heart_data_1.rename(columns={'id': 'Unnamed: 0', 'thalch': 'thalach'})

# Step 2: Define mappings for categorical variables
# Chest pain type (cp)
cp_mapping = {
    "typical angina": 1,
    "atypical angina": 2,
    "non-anginal": 3,
    "asymptomatic": 4
}

# Resting ECG results (restecg)
restecg_mapping = {
    "normal": 0,
    "lv hypertrophy": 1,
    "st-t abnormality": 2
}

# Slope of the peak exercise ST segment (slope)
slope_mapping = {
    "upsloping": 1,
    "flat": 2,
    "downsloping": 3
}

# Thalassemia (thal)
thal_mapping = {
    "normal": 3,
    "fixed defect": 6,
    "reversable defect": 7
}

# Mapping for sex
sex_mapping = {
    "Male": 1,
    "Female": 0
}

# Step 3: Apply mappings to transform heart_data_1
heart_data_1['cp'] = heart_data_1['cp'].map(cp_mapping)
heart_data_1['restecg'] = heart_data_1['restecg'].map(restecg_mapping)
heart_data_1['slope'] = heart_data_1['slope'].map(slope_mapping)
heart_data_1['thal'] = heart_data_1['thal'].map(thal_mapping)
heart_data_1['sex'] = heart_data_1['sex'].map(sex_mapping)

# Step 4: Convert boolean columns to integers (0 and 1)
heart_data_1['fbs'] = heart_data_1['fbs'].astype(float)
heart_data_1['exang'] = heart_data_1['exang'].astype(float)

# Step 5: Ensure columns in heart_data_1 match the columns in heart_data_2
heart_data_1 = heart_data_1[heart_data_2.columns]

# Step 6: Merge the two DataFrames and drop duplicates
merged_df = pd.concat([heart_data_1, heart_data_2], ignore_index=True)
merged_df = merged_df.drop_duplicates()

# Display the first few rows of the merged DataFrame
print(merged_df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'heart_disease_uci.csv'

In [28]:
# Step 5: Select only the columns present in heart_data_2 (after renaming)
heart_data_1 = heart_data_1[heart_data_2.columns]

# Step 6: Merge the two DataFrames and drop duplicates
merged_df = pd.concat([heart_data_1, heart_data_2], ignore_index=True)
merged_df = merged_df.drop_duplicates()

# Display the first few rows of the merged DataFrame
print(merged_df.head())

   Unnamed: 0  age     sex  cp  trestbps   chol  fbs  restecg  thalach  exang  \
0           1   63    Male   1     145.0  233.0  1.0      1.0    150.0    0.0   
1           2   67    Male   4     160.0  286.0  0.0      1.0    108.0    1.0   
2           3   67    Male   4     120.0  229.0  0.0      1.0    129.0    1.0   
3           4   37    Male   3     130.0  250.0  0.0      0.0    187.0    0.0   
4           5   41  Female   2     130.0  204.0  0.0      1.0    172.0    0.0   

   oldpeak  slope   ca  thal  num  
0      2.3    3.0  0.0   6.0    0  
1      1.5    2.0  3.0   3.0    2  
2      2.6    2.0  2.0   7.0    1  
3      3.5    3.0  0.0   3.0    0  
4      1.4    1.0  0.0   3.0    0  


In [29]:
merged_df.shape

(1223, 15)